In [25]:
import pandas as pd
import os, re, string
from langdetect import detect
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Checkout\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Checkout\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Import csv files dynamically
current_directory = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(current_directory, '..'))
job_posting = os.path.join(project_root, 'src', 'data', 'job_posting.csv')
students = os.path.join(project_root, 'src', 'data', 'students.csv')

In [6]:
job_posting = pd.read_csv(job_posting)
students = pd.read_csv(students)

In [7]:
job_posting.head()

,job_id,company_id,company_name,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,original_listed_time,remote_allowed,views,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,company_speciality,company_industry,company_description,linkedin_url,job_skills
0,3757940104,553718,HearingLife,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",NaN,1.699090e+12,NaN,9.0,https://www.linkedin.com/jobs/view/3757940104/...,https://careers-demant.icims.com/jobs/19601/he...,OffsiteApply,1.701680e+12,NaN,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,"Digital Hearing Instruments, behind ear, in ea...",Medical Equipment Manufacturing,HearingLife\u202fis a national hearing care co...,https://www.linkedin.com/company/hearing-life,Other
1,3757940025,2192142,"Metalcraft of Mayville, Inc.",Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",NaN,1.699080e+12,NaN,NaN,https://www.linkedin.com/jobs/view/3757940025/...,https://www.click2apply.net/mXLQz5S5NEYEXsKjwH...,OffsiteApply,1.701680e+12,NaN,NaN,NaN,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420,"Metal Fabrication, Machining, Robotic Welding,...",Industrial Machinery Manufacturing,"Headquartered in Mayville, Wisconsin, Metalcra...",https://www.linkedin.com/company/metalcraft-of...,"Management, Manufacturing"
2,3757938019,474443,"U.S. Tsubaki Power Transmission, LLC","Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",NaN,1.699080e+12,NaN,NaN,https://www.linkedin.com/jobs/view/3757938019/...,https://www.click2apply.net/LwbOykH2yAJdahB5Ah...,OffsiteApply,1.701680e+12,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644,"Roller Chain, Engineering Class Chain, Sprocke...",Automation Machinery Manufacturing,"U.S. Tsubaki Power Transmission, LLC is a subs...",https://www.linkedin.com/company/u.s.-tsubaki-...,Engineering
3,3757938018,18213359,Episcopal Communities & Services,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",NaN,1.699080e+12,NaN,1.0,https://www.linkedin.com/jobs/view/3757938018/...,https://jobs.apploi.com/view/854782?utm_campai...,OffsiteApply,1.701680e+12,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,NaN,Non-profit Organizations,Episcopal Communities & Services (ECS) has bec...,https://www.linkedin.com/company/episcopal-com...,"Management, Manufacturing"
4,3757937095,437225,"iHerb, LLC",Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,NaN,1.698970e+12,1.0,NaN,https://www.linkedin.com/jobs/view/3757937095/...,https://careers.iherb.com/global/en/job/IHINGL...,OffsiteApply,1.701680e+12,NaN,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346,"Vitamins, Sports Nutrition, Natural Supplement...","Retail, Manufacturing, Technology, Information...","""iHerb is on a mission to make health and well...",https://www.linkedin.com/company/iherb,Information Technology


In [8]:
job_posting.isnull().sum()

job_id                            0
company_id                        0
company_name                    705
title                             0
description                       1
max_salary                    22135
med_salary                    31005
min_salary                    22135
pay_period                    19894
formatted_work_type               0
location                          0
applies                       17008
original_listed_time              0
remote_allowed                28444
views                          7360
job_posting_url                   0
application_url               12250
application_type                  0
expiry                            0
closed_time                   32074
formatted_experience_level     9181
skills_desc                   32909
listed_time                       0
posting_domain                13558
sponsored                         0
work_type                         0
currency                      19894
compensation_type           

In [9]:
job_posting[job_posting['description'].isnull()]

,job_id,company_id,company_name,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,original_listed_time,remote_allowed,views,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,company_speciality,company_industry,company_description,linkedin_url,job_skills
26954,3694104501,157262,First Horizon Bank,Senior Universal Banker,NaN,NaN,NaN,NaN,NaN,Full-time,Nashville Metropolitan Area,NaN,1.692840e+12,NaN,1.0,https://www.linkedin.com/jobs/view/3694104501/...,NaN,SimpleOnsiteApply,1.695430e+12,NaN,NaN,NaN,1.692840e+12,NaN,0,FULL_TIME,NaN,NaN,1,NaN,Financial Services,When we opened our doors in 1864 on North Cour...,https://www.linkedin.com/company/first-horizon...,"Finance, Sales"


In [10]:
# if description is empty fill it with company_description, company_industry and job_skills
job_posting['description'] = job_posting['description'].fillna(job_posting['company_description'] + ' ' + job_posting['company_industry'] + ' ' + job_posting['job_skills'])

In [11]:
contraction_mapping = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "this's": "this is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "here's": "here is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
    "u.s": "america",
    "e.g": "for example",
}

In [12]:
# Clean contraction
def clean_contractions(text):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in contraction_mapping.keys():
        if "" + word + "" in text:
            text = text.replace("" + word + "", "" + contraction_mapping[word] + "")
    return text

In [13]:
# Remove Url Pattern
def remove_urls(text):
    url_pattern = r"https?://\S+|www\.\S+"
    return re.sub(url_pattern, "", text)

In [14]:
# Remove HTML Tag
def remove_html(text):
    html_pattern = re.compile("<.*?>")
    return html_pattern.sub(r"", text)

In [15]:
# Remove special character "!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~"
def remove_punctuation(text):
    punctuation = string.punctuation + "–"
    return re.sub(f"[{re.escape(punctuation)}]", "", text)

In [16]:
# Remove E-mail pattern
def remove_emails(text):
    return re.sub(r"\S+@\S+", "", text)

In [17]:
# Remove New Line Code Snippet
def remove_code_snippet(text):
    return text.replace("\n", "")

In [18]:
# Remove Emoji
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

In [19]:
# Remove Non-English Text
def remove_non_english_text(text):
    def is_english(text):
        try:
            return detect(text) == "en"
        except:
            return False

    return text if is_english(text) else ""

In [20]:
# Remove Digits
def remove_digits(text):
    return "".join(filter(lambda char: not char.isdigit(), text))

In [21]:
# Remove Stop Words
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    custom_stop_words = [
        "job",
        "role",
        "position",
        "responsibility",
        "responsibilities",
        "duties",
        "duty",
        "requirement",
        "requirements",
        "qualification",
        "qualifications",
        "description",
        "descriptions",
        "candidate",
        "candidates",
        "applicant",
        "applicants",
        "opportunity",
        "opportunities",
        "team",
        "teams",
        "work",
        "working",
        "employee",
        "employees",
        "employer",
        "employers",
        "company",
        "companies",
        "location",
        "locations",
        "department",
        "departments",
        "report",
        "reports",
        "reporting",
        "benefit",
        "benefits",
        "compensation",
        "salary",
        "experience",
        "experienced",
        "year",
        "years",
        "gender",
        "race",
        "color",
        "sex",
        "orientation",
        "sexual",
        "religion",
        "national",
        "identify",
        "veteran",
        "nation",
        "including",
        "required",
        "disability",
        "regard"
    ]

In [22]:
# Remove Stop Words
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    custom_stop_words = [
        "job",
        "role",
        "position",
        "responsibility",
        "responsibilities",
        "duties",
        "duty",
        "requirement",
        "requirements",
        "qualification",
        "qualifications",
        "description",
        "descriptions",
        "candidate",
        "candidates",
        "applicant",
        "applicants",
        "opportunity",
        "opportunities",
        "team",
        "teams",
        "work",
        "working",
        "employee",
        "employees",
        "employer",
        "employers",
        "company",
        "companies",
        "location",
        "locations",
        "department",
        "departments",
        "report",
        "reports",
        "reporting",
        "benefit",
        "benefits",
        "compensation",
        "salary",
        "experience",
        "experienced",
        "year",
        "years",
        "gender",
        "race",
        "color",
        "sex",
        "orientation",
        "sexual",
        "religion",
        "national",
        "identify",
        "veteran",
        "nation",
        "including",
        "required",
        "disability",
        "regard"
    ]

    words = text.split()
    filtered_words = [
        word
        for word in words
        if word.lower() not in stop_words and word.lower() not in custom_stop_words
    ]
    return " ".join(filtered_words)

In [23]:
# Lemmatization
def lemmatize_words(text):
    words = text.split()
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

In [26]:
def clean_text(text,remove_stop_words=True):
    text = text.lower()
    text = clean_contractions(text)
    text = remove_urls(text)
    text = remove_html(text)
    text = remove_code_snippet(text)
    text = remove_punctuation(text)
    text = remove_emails(text)
    text = remove_emoji(text)
    text = remove_non_english_text(text)
    text = remove_digits(text)
    text = lemmatize_words(text)
    if remove_stop_words:
        text = remove_stopwords(text)
    return text

In [27]:
#Clean Job description text
job_posting['description_cleaned'] = job_posting['description'].astype(str).apply(lambda x: clean_text(x))
job_posting['description_cleaned_st'] = job_posting['description'].astype(str).apply(lambda x: clean_text(x, remove_stop_words=False))

In [28]:
job_posting.head()

,job_id,company_id,company_name,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,original_listed_time,remote_allowed,views,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,scraped,company_speciality,company_industry,company_description,linkedin_url,job_skills,description_cleaned,description_cleaned_st
0,3757940104,553718,HearingLife,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",NaN,1.699090e+12,NaN,9.0,https://www.linkedin.com/jobs/view/3757940104/...,https://careers-demant.icims.com/jobs/19601/he...,OffsiteApply,1.701680e+12,NaN,Entry level,NaN,1.699090e+12,careers-demant.icims.com,0,FULL_TIME,USD,BASE_SALARY,1699138101,"Digital Hearing Instruments, behind ear, in ea...",Medical Equipment Manufacturing,HearingLife\u202fis a national hearing care co...,https://www.linkedin.com/company/hearing-life,Other,overviewhearinglife hearing care part demant g...,overviewhearinglife is a national hearing care...
1,3757940025,2192142,"Metalcraft of Mayville, Inc.",Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,NaN,NaN,NaN,NaN,Full-time,"Beaver Dam, WI",NaN,1.699080e+12,NaN,NaN,https://www.linkedin.com/jobs/view/3757940025/...,https://www.click2apply.net/mXLQz5S5NEYEXsKjwH...,OffsiteApply,1.701680e+12,NaN,NaN,NaN,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085420,"Metal Fabrication, Machining, Robotic Welding,...",Industrial Machinery Manufacturing,"Headquartered in Mayville, Wisconsin, Metalcra...",https://www.linkedin.com/company/metalcraft-of...,"Management, Manufacturing",metalcraft mayvillemetalcraft mayville equal e...,metalcraft of mayvillemetalcraft of mayville i...
2,3757938019,474443,"U.S. Tsubaki Power Transmission, LLC","Manager, Engineering",\nThe TSUBAKI name is synonymous with excellen...,NaN,NaN,NaN,NaN,Full-time,"Bessemer, AL",NaN,1.699080e+12,NaN,NaN,https://www.linkedin.com/jobs/view/3757938019/...,https://www.click2apply.net/LwbOykH2yAJdahB5Ah...,OffsiteApply,1.701680e+12,NaN,NaN,Bachelor's Degree in Mechanical Engineering pr...,1.699080e+12,www.click2apply.net,0,FULL_TIME,NaN,NaN,1699085644,"Roller Chain, Engineering Class Chain, Sprocke...",Automation Machinery Manufacturing,"U.S. Tsubaki Power Transmission, LLC is a subs...",https://www.linkedin.com/company/u.s.-tsubaki-...,Engineering,tsubaki name synonymous excellence quality dep...,the tsubaki name is synonymous with excellence...
3,3757938018,18213359,Episcopal Communities & Services,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",NaN,1.699080e+12,NaN,1.0,https://www.linkedin.com/jobs/view/3757938018/...,https://jobs.apploi.com/view/854782?utm_campai...,OffsiteApply,1.701680e+12,NaN,Entry level,NaN,1.699080e+12,jobs.apploi.com,0,FULL_TIME,USD,BASE_SALARY,1699087461,NaN,Non-profit Organizations,Episcopal Communities & Services (ECS) has bec...,https://www.linkedin.com/company/episcopal-com...,"Management, Manufacturing",descriptiontitle looking great develop profess...,descriptiontitle looking for a great opportuni...
4,3757937095,437225,"iHerb, LLC",Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,NaN,1.698970e+12,1.0,NaN,https://www.linkedin.com/jobs/view/3757937095/...,https://careers.iherb.com/global/en/job/IHINGL...,OffsiteApply,1.701680e+12,NaN,Mid-Senior level,NaN,1.699090e+12,careers.iherb.com,0,FULL_TIME,USD,BASE_SALARY,1699085346,"Vitamins, Sports Nutrition, Natural Supplement...","Retail, Manufacturing, Technology, Information...","""iHerb is on a mission to make health and well...",https://www.linkedin.com/company/iherb,Information Technology,summaryat iherb mission make health 

In [31]:
# you can run this cell to get final job_posting file
job_posting.to_csv('job_posting_description_cleaned.csv', index=False)